In [1]:
import copy
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

from tqdm.notebook import tqdm
from sklearn.model_selection import KFold 


### Load the preprocessed Aligned corpus (includes surprisal and incremental information value estimates)

In [ ]:
aligned_norm = pd.read_csv("preprocessed_corpora/euclidean/aligned_preprocessed_normalised.csv")

# aligned.columns = aligned.columns.str.replace("-", "_")
aligned_norm.columns = aligned_norm.columns.str.replace("-", "_")


In [3]:
# For interaction terms

def all_pairs(list):
    """
    Returns all possible pairs of elements in a list
    """
    pairs = []
    for i in range(len(list)):
        for j in range(i+1, len(list)):
            pairs.append([list[i], list[j]])
    return pairs

In [4]:
# Constants

MODEL_NAMES = ['gpt2_small', 'gpt2_medium', 'gpt2_large', 'gpt2_xl']

LAYERS = {
    "gpt2_small": list(range(0, 13)),
    "gpt2_medium": list(range(0, 25, 2)),
    "gpt2_large": list(range(0, 37, 3)),
    "gpt2_xl": list(range(0, 49, 4))
}

RATINGS = ['rating_mean', 'rating_s', 'rating_sd', 'cloze_p_smoothed', 'cloze_s', 'competition', 'entropy']
ERP = ['ELAN', 'LAN', 'N400', 'EPNP', 'P600', 'PNP']
RT = ['RTfirstfix', 'RTfirstpass', 'RTrightbound', 'RTgopast', 'self_paced_reading_time']
ALL_PREDICTED_VARIABLES = RATINGS + RT + ERP

BASELINE_PREDICTORS = ['Subtlex_log10', 'context_length', 'length']
SURPRISAL_PREDICTORS = [col for col in aligned_norm if '_surprisal' in col]
IV_PREDICTORS = [col for col in aligned_norm if '_iv_' in col]
ALL_INFORMATION_PREDICTORS = SURPRISAL_PREDICTORS + IV_PREDICTORS 


### Linear models: target vs baseline

In [5]:
# this can take a while... (runtime: 9m 10s)


predicted_variables = ALL_PREDICTED_VARIABLES
baseline_predictors = BASELINE_PREDICTORS
all_predictors = ALL_INFORMATION_PREDICTORS


baseline_predictors_str = " + ".join(baseline_predictors) + " + " + " + ".join([f"{p[0]}:{p[1]}" for p in all_pairs(baseline_predictors)])
    
results_control_baseline_aligned = []


for predicted_var in tqdm(predicted_variables):

    df_tmp = aligned_norm[[predicted_var] + baseline_predictors + all_predictors].dropna()
    
    # ------------------------------------------
    # ANOVA for surprisal and IAS models
    # ------------------------------------------

    # first fit baseline model
    OLS_baseline = smf.ols(
        formula=f'{predicted_var} ~ {baseline_predictors_str}', 
        data=df_tmp
    ).fit()
    
    surprisal_ols_models = {}
    
    # fit surprisal models
    for model in MODEL_NAMES:
        predictor = f"{model}_surprisal"
        OLS_model = smf.ols(
            formula=f'{predicted_var} ~ {baseline_predictors_str} + {predictor}', 
            data=df_tmp
        ).fit()
        surprisal_ols_models[model] = copy.deepcopy(OLS_model)

        anova_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
        results_control_baseline_aligned.append({
            "y": predicted_var, 
            "metric": "Surprisal", 
            "model": model, 
            "layer": "",
            "layer_idx": "",
            "horizon": "",
            "aggregation": "",
            "fold": "full", 
            "rsquared": OLS_model.rsquared,
            "rsquared_adj": OLS_model.rsquared_adj,
            "aic": OLS_model.aic,
            "bic": OLS_model.bic,
            "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
            "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
            "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
            "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
            "anova_baseline_rss": anova_results.ssr[1],
            "anova_baseline_delta_ss": anova_results.ss_diff[1],
            "anova_baseline_p": anova_results['Pr(>F)'][1],
            "anova_surprisal_rss": np.nan,
            "anova_surprisal_delta_ss": np.nan,
            "anova_surprisal_p": np.nan
        })

    
    # fit IAS models by aggregation function
    for model in MODEL_NAMES:
        for agg_func in ["Smean", "Smin", "Smax"]:
            predictors = [p for p in IV_PREDICTORS if p.startswith(model) and p.endswith(agg_func)]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()

            anova_baseline_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            anova_surprisal_results = sm.stats.anova_lm(surprisal_ols_models[model], OLS_model)

            results_control_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": "All",
                "layer_idx": "All",
                "horizon": "All",
                "aggregation": agg_func[1:],
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_baseline_rss": anova_baseline_results.ssr[1],
                "anova_baseline_delta_ss": anova_baseline_results.ss_diff[1],
                "anova_baseline_p": anova_baseline_results['Pr(>F)'][1],
                "anova_surprisal_rss": anova_surprisal_results.ssr[1],
                "anova_surprisal_delta_ss": anova_surprisal_results.ss_diff[1],
                "anova_surprisal_p": anova_surprisal_results['Pr(>F)'][1]
            })

    # fit IAS models by layer
    for model in MODEL_NAMES:
        for layer_idx, layer in enumerate(LAYERS[model]):

            predictors = [p for p in IV_PREDICTORS if f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()

            anova_baseline_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            anova_surprisal_results = sm.stats.anova_lm(surprisal_ols_models[model], OLS_model)

            results_control_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": layer,
                "layer_idx": layer_idx,
                "horizon": "All",
                "aggregation": "mean",
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_baseline_rss": anova_baseline_results.ssr[1],
                "anova_baseline_delta_ss": anova_baseline_results.ss_diff[1],
                "anova_baseline_p": anova_baseline_results['Pr(>F)'][1],
                "anova_surprisal_rss": anova_surprisal_results.ssr[1],
                "anova_surprisal_delta_ss": anova_surprisal_results.ss_diff[1],
                "anova_surprisal_p": anova_surprisal_results['Pr(>F)'][1]
            })

    # fit IAS models by horizon
    for model in MODEL_NAMES:
        for horizon in range(1, 11):
            predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and p.startswith(model) and p.endswith("Smean")]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()

            anova_baseline_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            anova_surprisal_results = sm.stats.anova_lm(surprisal_ols_models[model], OLS_model)

            results_control_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": "All",
                "layer_idx": "All",
                "horizon": horizon,
                "aggregation": "mean",
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_baseline_rss": anova_baseline_results.ssr[1],
                "anova_baseline_delta_ss": anova_baseline_results.ss_diff[1],
                "anova_baseline_p": anova_baseline_results['Pr(>F)'][1],
                "anova_surprisal_rss": anova_surprisal_results.ssr[1],
                "anova_surprisal_delta_ss": anova_surprisal_results.ss_diff[1],
                "anova_surprisal_p": anova_surprisal_results['Pr(>F)'][1]
            })
        
    # fit IAS models by horizon-layer combination
    for model in MODEL_NAMES:
        for layer_idx, layer in enumerate(LAYERS[model]):
            for horizon in range(1, 11):
                predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]
                
                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                    data=df_tmp
                ).fit()
                
                anova_baseline_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
                anova_surprisal_results = sm.stats.anova_lm(surprisal_ols_models[model], OLS_model)
                
#                 print(anova_surprisal_results)
                
                results_control_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": layer,
                    "layer_idx": layer_idx,
                    "horizon": horizon,
                    "aggregation": "mean",
                    "fold": "full", 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_baseline_rss": anova_baseline_results.ssr[1],
                    "anova_baseline_delta_ss": anova_baseline_results.ss_diff[1],
                    "anova_baseline_p": anova_baseline_results['Pr(>F)'][1],
                    "anova_surprisal_rss": anova_surprisal_results.ssr[1],
                    "anova_surprisal_delta_ss": anova_surprisal_results.ss_diff[1],
                    "anova_surprisal_p": anova_surprisal_results['Pr(>F)'][1]
                })
    
    # ---------------------------------------------------
    # 10-fold bootstrapping for surprisal and IAS models
    # ---------------------------------------------------
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(df_tmp) 

    for fold, (split_indices, _) in enumerate(kf.split(df_tmp)):
        df_tmp_fold = df_tmp.iloc[split_indices]

        # first fit baseline model without predictability
        OLS_baseline = smf.ols(
            formula=f'{predicted_var} ~ {baseline_predictors_str}', 
            data=df_tmp_fold
        ).fit()

        # fit surprisal models
        for model in MODEL_NAMES:
            predictor = f"{model}_surprisal"
            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {predictor}', 
                data=df_tmp_fold
            ).fit()

            results_control_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Surprisal", 
                "model": model, 
                "layer": "",
                "layer_idx": "",
                "horizon": "",
                "aggregation": "",
                "fold": fold, 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_baseline_rss": "",
                "anova_baseline_delta_ss": "",
                "anova_baseline_p": "",
                "anova_surprisal_rss": "",
                "anova_surprisal_delta_ss": "",
                "anova_surprisal_p": "",
            })


        # fit IAS models by aggregation function
        for model in MODEL_NAMES:
            for agg_func in ["Smean", "Smin", "Smax"]:
                predictors = [p for p in IV_PREDICTORS if p.startswith(model) and p.endswith(agg_func)]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()

                results_control_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": "All",
                    "layer_idx": "All",
                    "horizon": "All",
                    "aggregation": agg_func[1:],
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_baseline_rss": "",
                    "anova_baseline_delta_ss": "",
                    "anova_baseline_p": "",
                    "anova_surprisal_rss": "",
                    "anova_surprisal_delta_ss": "",
                    "anova_surprisal_p": "",
                })


        # fit IAS models by layer
        for model in MODEL_NAMES:
            for layer_idx, layer in enumerate(LAYERS[model]):

                predictors = [p for p in IV_PREDICTORS if f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()
                results_control_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": layer,
                    "layer_idx": layer_idx,
                    "horizon": "All",
                    "aggregation": "mean",
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_baseline_rss": "",
                    "anova_baseline_delta_ss": "",
                    "anova_baseline_p": "",
                    "anova_surprisal_rss": "",
                    "anova_surprisal_delta_ss": "",
                    "anova_surprisal_p": "",
                })

        # fit IAS models by horizon
        for model in MODEL_NAMES:
            for horizon in range(1, 11):
                predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and p.startswith(model) and p.endswith("Smean")]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()
                results_control_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": "All",
                    "layer_idx": "All",
                    "horizon": horizon,
                    "aggregation": "mean",
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_baseline_rss": "",
                    "anova_baseline_delta_ss": "",
                    "anova_baseline_p": "",
                    "anova_surprisal_rss": "",
                    "anova_surprisal_delta_ss": "",
                    "anova_surprisal_p": "",
                })

       
results_control_baseline_aligned_df = pd.DataFrame(results_control_baseline_aligned)


  0%|          | 0/18 [00:00<?, ?it/s]

Save output of linear models: surprisal and IAS against control baseline.

In [ ]:
results_control_baseline_aligned_df.to_csv(
    "results/euclidean/aligned_ols_against_baseline.csv",
    index=False
)

### Linear models: information value vs surprisal baseline

In [8]:
# this can take a while... (runtime: 7m 25s)


predicted_variables = ALL_PREDICTED_VARIABLES
all_predictors = ALL_INFORMATION_PREDICTORS

baseline_predictors = BASELINE_PREDICTORS
baseline_predictors_str = " + ".join(baseline_predictors) + " + " + " + ".join([f"{p[0]}:{p[1]}" for p in all_pairs(baseline_predictors)])
    
    
results_surprisal_baseline_aligned = []

for predicted_var in tqdm(predicted_variables):

    df_tmp = aligned_norm[[predicted_var] + baseline_predictors + all_predictors].dropna()

    for model in MODEL_NAMES:
        
        # ------------------------------------------
        # ANOVA for IAS models
        # ------------------------------------------
        
        # first fit baseline model including surprisal
        OLS_baseline = smf.ols(
            formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal', 
            data=df_tmp
        ).fit()        
    
        # fit IAS models by aggregation function
        for agg_func in ["Smean", "Smin", "Smax"]:
            predictors = [p for p in IV_PREDICTORS if p.startswith(model) and p.endswith(agg_func)]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()
            
            anova_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            
            results_surprisal_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": "All",
                "layer_idx": "All",
                "horizon": "All",
                "aggregation": agg_func[1:],
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_rss": anova_results.ssr[1],
                "anova_delta_ss": anova_results.ss_diff[1],
                "anova_p": anova_results['Pr(>F)'][1]
            })
    
        # fit IAS models by layer
        for layer_idx, layer in enumerate(LAYERS[model]):
        
            predictors = [p for p in IV_PREDICTORS if f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()
            
            anova_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            
            results_surprisal_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": layer,
                "layer_idx": layer_idx,
                "horizon": "All",
                "aggregation": "mean",
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_rss": anova_results.ssr[1],
                "anova_delta_ss": anova_results.ss_diff[1],
                "anova_p": anova_results['Pr(>F)'][1]
            })
    
        # fit IAS models by horizon
        for horizon in range(1, 11):
            predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and p.startswith(model) and p.endswith("Smean")]

            OLS_model = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                data=df_tmp
            ).fit()
            
            anova_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
            
            results_surprisal_baseline_aligned.append({
                "y": predicted_var, 
                "metric": "Information value", 
                "model": model, 
                "layer": "All",
                "layer_idx": "All",
                "horizon": horizon,
                "aggregation": "mean",
                "fold": "full", 
                "rsquared": OLS_model.rsquared,
                "rsquared_adj": OLS_model.rsquared_adj,
                "aic": OLS_model.aic,
                "bic": OLS_model.bic,
                "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                "anova_rss": anova_results.ssr[1],
                "anova_delta_ss": anova_results.ss_diff[1],
                "anova_p": anova_results['Pr(>F)'][1],
            })
            
        # fit IAS models by horizon-layer combination
        for layer_idx, layer in enumerate(LAYERS[model]):
            for horizon in range(1, 11):
                predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                    data=df_tmp
                ).fit()
                
                anova_results = sm.stats.anova_lm(OLS_baseline, OLS_model)
                
                results_surprisal_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": layer,
                    "layer_idx": layer_idx,
                    "horizon": horizon,
                    "aggregation": "mean",
                    "fold": "full", 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_rss": anova_results.ssr[1],
                    "anova_delta_ss": anova_results.ss_diff[1],
                    "anova_p": anova_results['Pr(>F)'][1],
                })
        
    
        # ---------------------------------------------------
        # 10-fold bootstrapping for IAS models
        # ---------------------------------------------------
        kf = KFold(n_splits=10, random_state=42, shuffle=True)
        kf.get_n_splits(df_tmp) 

        for fold, (split_indices, _) in enumerate(kf.split(df_tmp)):
            df_tmp_fold = df_tmp.iloc[split_indices]

            # first fit baseline model including surprisal
            OLS_baseline = smf.ols(
                formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal', 
                data=df_tmp_fold
            ).fit()     

            # fit IAS models by aggregation function
            for agg_func in ["Smean", "Smin", "Smax"]:
                predictors = [p for p in IV_PREDICTORS if p.startswith(model) and p.endswith(agg_func)]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()

                results_surprisal_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": "All",
                    "layer_idx": "All",
                    "horizon": "All",
                    "aggregation": agg_func[1:],
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_rss": "",
                    "anova_delta_ss": "",
                    "anova_p": "",
                })


            # fit IAS models by layer
            for layer_idx, layer in enumerate(LAYERS[model]):

                predictors = [p for p in IV_PREDICTORS if f"_L{layer}_" in p and p.startswith(model) and p.endswith("Smean")]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()
                results_surprisal_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": layer,
                    "layer_idx": layer_idx,
                    "horizon": "All",
                    "aggregation": "mean",
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_rss": "",
                    "anova_delta_ss": "",
                    "anova_p": ""
                })

            # fit IAS models by horizon
            for horizon in range(1, 11):
                predictors = [p for p in IV_PREDICTORS if f"_H{horizon}_" in p and p.startswith(model) and p.endswith("Smean")]

                OLS_model = smf.ols(
                    formula=f'{predicted_var} ~ {baseline_predictors_str} + {model}_surprisal + {"+".join(predictors)}', 
                    data=df_tmp_fold
                ).fit()
                results_surprisal_baseline_aligned.append({
                    "y": predicted_var, 
                    "metric": "Information value", 
                    "model": model, 
                    "layer": "All",
                    "layer_idx": "All",
                    "horizon": horizon,
                    "aggregation": "mean",
                    "fold": fold, 
                    "rsquared": OLS_model.rsquared,
                    "rsquared_adj": OLS_model.rsquared_adj,
                    "aic": OLS_model.aic,
                    "bic": OLS_model.bic,
                    "delta_rsquared": OLS_model.rsquared - OLS_baseline.rsquared, 
                    "delta_rsquared_adj": OLS_model.rsquared_adj - OLS_baseline.rsquared_adj, 
                    "delta_rsquared_aic": OLS_model.aic - OLS_baseline.aic,
                    "delta_rsquared_bic": OLS_model.bic - OLS_baseline.bic,
                    "anova_rss": "",
                    "anova_delta_ss": "",
                    "anova_p": ""
                })

       
results_surprisal_baseline_aligned_df = pd.DataFrame(results_surprisal_baseline_aligned)


  0%|          | 0/18 [00:00<?, ?it/s]

Save output of linear models: IAS against surprisal baseline.

In [ ]:
results_surprisal_baseline_aligned_df.to_csv(
    "results/euclidean/aligned_ols_against_surprisal.csv",
    index=False
)